In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.baseline_rank.baseline_rank import BaselineRank
from scripts.models.ltr.train import LTRTrain
from scripts.extractive_summary.ltr.ltr_features_targets import LTRFeaturesTargets
from scripts.models.ltr_gbt.train import LTRGBTTrain
from scripts.models.rank_with_model.metrics import RankModelMetrics
from scripts import conf
%reload_ext autoreload

In [4]:
import pandas as pd

# Evaluación

Se usará el mismo desarrollo que se utilizó cuando se calcularon las métricas de los resúmenes generados por eventos clave y por grafos
conceptuales

# Baseline

## Carga de datos

In [5]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'cosine_tfidf'
drop_teams = True
lemma = True
metric_params = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [6]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to cosine_tfidf


In [7]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [23]:
rank.run()

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv


In [24]:
rank.ltr.target_metric

'cosine_tfidf'

## Evaluación

In [11]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [12]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

Los vocab size representan el número de frases que hay en la referencia. 

In [ ]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=False)

### Con procesado

In [13]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/baseline_rank_cosine_tfidf_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/baseline_rank_cosine_tfidf_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/baseline_rank_cosine_tfidf_processed_avg.pickle


In [14]:
avg_scores_dict

8.631958094422498e-05

# Random forest

## Carga de datos

In [30]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}

num_features = ['tfidf_sum']
cat_features_dict = {'is_key_event': [0, 1],
                     'n_players_cat': ['no_player', 'one_player', 'more_than_one_player']}

model_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [2, 5, 7],
    'bootstrap': [True],
    
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [31]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to rouge


In [32]:
#CV
ltr_train = LTRTrain(cat_features_dict=cat_features_dict,
                     num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to rouge


In [33]:
ltr_train.experiment_id()

'4649644395'

In [34]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


## Evaluación

In [35]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [36]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

In [37]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                        preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/1e1b5e1354/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_rouge-1_r_4649644395
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_rouge-1_r_4649644395.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_avg.pickle


In [38]:
avg_scores_dict

0.0001298055626846204

### Con procesado

In [39]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/1e1b5e1354/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_processed_avg.pickle


In [40]:
avg_scores_dict

6.759536855045942e-05

# GBT

## Carga de datos

In [66]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}

model_params = {
    'min_samples_leaf': [50],
    'subsample': [0.8],  
    'learning_rate': [0.1],
    'n_estimators': [80],
    'max_depth':range(5,16,4),
    'min_samples_split': range(200,1001,300)
}
cv = 5

In [67]:
ltr_train = LTRGBTTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to rouge


In [68]:
ltr_train.experiment_id()

'be3b14bd22'

In [69]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840'

## Evaluación

In [70]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [71]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

In [72]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/02cf14e54e/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22_avg.pickle


In [73]:
avg_scores_dict

0.00013162836111063557

### Con procesado

In [74]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/02cf14e54e/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22_processed_avg.pickle


In [75]:
avg_scores_dict

6.835684273896406e-05

# Eventos clave y grafos conceptuales

In [6]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [7]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


In [8]:
%%time
evaluation.evaluate_all_summaries(preprocess_text=True)

Evaluating the following summaries: ['key_events_summaries_1.csv', 'key_events_summaries_3.csv', 'key_events_summaries_graph_7.csv', 'key_events_summaries_4.csv', 'key_events_summaries_graph_2.csv', 'key_events_summaries_graph_8.csv', 'key_events_summaries_graph_6.csv', 'key_events_summaries_graph_3.csv', 'key_events_summaries_2.csv', 'key_events_summaries_graph_5.csv', 'key_events_summaries_graph_9.csv', 'key_events_summaries_graph_1.csv', 'key_events_summaries_graph_4.csv']
Evaluating key_events_summaries_1.csv
/home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_1_processed
Metrics already exist
Evaluating key_events_summaries_3.csv
/home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_3_processed
Metrics already exist
Evaluating key_events_summaries_graph_7.csv
/home/carlos/MasterDS/tfm/data/metrics/summaries/sms/key_events_summaries_graph_7_processed
There are 1 articles with an empty summary
Performing evaluation for 4498 articles
Writing 

# Todos los resúmenes

In [76]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [77]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


In [78]:
pd_metrics = evaluation.output_avg_metrics()

In [79]:
pd_metrics.set_index('experiment').sort_index()

,score
experiment,
baseline_rank_cosine_tfidf,0.000372
baseline_rank_cosine_tfidf_processed,0.000086
key_events_summaries_1,0.000399
key_events_summaries_1_processed,0.000087
key_events_summaries_2,0.000397
key_events_summaries_2_processed,0.000087
key_events_summaries_3,0.000399
key_events_summaries_3_processed,0.000087
key_events_summaries_4,0.000397


In [10]:
pd_metrics.sort_values('score')

,experiment,score
4,rank_with_model_ltr_random_forest_cosine_emb_1...,0.000068
8,rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80_...,0.000069
14,baseline_rank_cosine_tfidf_processed,0.000086
2,key_events_summaries_graph_2,0.000210
7,key_events_summaries_graph_4,0.000210
11,rank_with_model_ltr_random_forest_cosine_emb_1...,0.000249
9,rank_with_model_ltr_gbt_cosine_emb_fca8c8bb80,0.000250
1,key_events_summaries_graph_1,0.000265
3,key_events_summaries_graph_3,0.000265
13,key_events_summaries_graph_5,0.000271
